## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Загрузим файл с данными и изучим общую информацию. 

In [1]:
import pandas as pd
solvency = pd.read_csv(r'C:\Users\Айболит\Desktop\DataFrames\projects\best_tariff_recomenfation\data.csv')
print(solvency.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Имеются пропуски в двух столбцах - общий стаж и ежемесячный доход. Возможно должностное лицо не имело этих данных и поэтому значения ячеек пусты, возможно, что эти люди не имеют стажа и дохода (тогда как банк решился выдать им кредит?) и их нужно заполнить числом близким к нулю, чтобы не исказил общие данные и, в то же время, с ними можно было работать или средним по категории населения (ежемесячный доход) и по возрастной категории (общий стаж).

In [2]:
print(solvency.columns)

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


Столбцы названы корректно.

In [3]:
solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Видим нарушение регистра в строках с типом object, числовые данные все должны быть строго больше нуля, тут имеются отрицательные значения.

In [4]:
solvency_none_values = solvency[solvency.isna().any(axis=1)]
print(solvency_none_values)

       children  days_employed  dob_years education  education_id  \
12            0            NaN         65   среднее             1   
26            0            NaN         41   среднее             1   
29            0            NaN         63   среднее             1   
41            0            NaN         50   среднее             1   
55            0            NaN         54   среднее             1   
...         ...            ...        ...       ...           ...   
21489         2            NaN         47   Среднее             1   
21495         1            NaN         50   среднее             1   
21497         0            NaN         48    ВЫСШЕЕ             0   
21502         1            NaN         42   среднее             1   
21510         2            NaN         28   среднее             1   

               family_status  family_status_id gender  income_type  debt  \
12          гражданский брак                 1      M    пенсионер     0   
26           женат 

Мы видим, что пропущены значения не случайно, а исключительно в рамках одного клиента. Мне нужно уточнить у должностного лица, приславшего мнеданные, причину пропуска значений, или посоветоваться со старшим товарищем - стоит ли заменять эти данные на числа близкие к нулю или на средние значения.

In [5]:
solvency['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

видим значение "0", которого в столбеце"возраст клиента" быть не должно.

In [6]:
print(solvency.duplicated().sum())

54


In [7]:
solvency['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

20 детей? Скорее всего это опечатка и хотели написать = 2. Также не может быть -1 ребенок, минус скорее всего попал случайно. Поэтому заменим 20 на 2, а -1 на 1.

In [8]:
solvency['children'].replace(-1, 1, inplace = True)
solvency['children'].replace(20, 2, inplace = True)

### Вывод

### Шаг 2. Предобработка данных

### Обработка пропусков

In [9]:
solvency ['education'] = solvency['education'].str.lower()

In [10]:
solvency['family_status'] = solvency['family_status'].str.lower()

In [11]:
solvency['income_type'] = solvency['income_type'].str.lower()

In [12]:
solvency['purpose'] = solvency['purpose'].str.lower()

In [13]:
solvency['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [14]:
insome_type_salary = solvency.loc[:,['income_type', 'total_income']]
print(insome_type_salary)


      income_type   total_income
0       сотрудник  253875.639453
1       сотрудник  112080.014102
2       сотрудник  145885.952297
3       сотрудник  267628.550329
4       пенсионер  158616.077870
...           ...            ...
21520   компаньон  224791.862382
21521   пенсионер  155999.806512
21522   сотрудник   89672.561153
21523   сотрудник  244093.050500
21524   сотрудник   82047.418899

[21525 rows x 2 columns]


создал таблицу для расчёта средних значений, чтобы не изменять общую в случае ошибок.

In [15]:
insome_type_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   income_type   21525 non-null  object 
 1   total_income  19351 non-null  float64
dtypes: float64(1), object(1)
memory usage: 336.5+ KB


In [16]:
worker_table = insome_type_salary[insome_type_salary['income_type']=='сотрудник']
print(worker_table)

      income_type   total_income
0       сотрудник  253875.639453
1       сотрудник  112080.014102
2       сотрудник  145885.952297
3       сотрудник  267628.550329
7       сотрудник  135823.934197
...           ...            ...
21515   сотрудник  109486.327999
21519   сотрудник  115949.039788
21522   сотрудник   89672.561153
21523   сотрудник  244093.050500
21524   сотрудник   82047.418899

[11119 rows x 2 columns]


In [17]:
worker_mean = worker_table['total_income'].mean()
print(worker_mean)

161380.26048788536


In [18]:
companion_table = insome_type_salary[insome_type_salary['income_type']=='компаньон']
print(companion_table)

      income_type   total_income
5       компаньон  255763.565419
6       компаньон  240525.971920
10      компаньон  113943.491460
14      компаньон  165127.911772
33      компаньон  157245.786233
...           ...            ...
21512   компаньон  147301.457769
21514   компаньон  355988.407188
21516   компаньон  322807.776603
21517   компаньон  178059.553491
21520   компаньон  224791.862382

[5085 rows x 2 columns]


In [19]:
companion_mean = companion_table['total_income'].mean()
print(companion_mean)

202417.46146177707


In [20]:
pensioner_table = insome_type_salary[insome_type_salary['income_type']=='пенсионер']
print(pensioner_table)

      income_type   total_income
4       пенсионер  158616.077870
12      пенсионер            NaN
18      пенсионер   56823.777243
24      пенсионер  290547.235997
25      пенсионер   55112.757732
...           ...            ...
21505   пенсионер   75439.993167
21508   пенсионер   72638.590915
21509   пенсионер   73029.059379
21518   пенсионер  153864.650328
21521   пенсионер  155999.806512

[3856 rows x 2 columns]


In [21]:
pensioner_mean = pensioner_table['total_income'].mean()
print(pensioner_mean)

137127.46569016558


In [22]:
servant_table = insome_type_salary[insome_type_salary['income_type']=='госслужащий']
print(servant_table)

       income_type   total_income
26     госслужащий            NaN
41     госслужащий            NaN
47     госслужащий  356277.909345
62     госслужащий  435388.195272
70     госслужащий  207561.466998
...            ...            ...
21288  госслужащий  233316.781101
21349  госслужащий   81624.022602
21380  госслужащий  103028.288433
21459  госслужащий  150568.114869
21507  госслужащий   98180.279152

[1459 rows x 2 columns]


In [23]:
servant_mean = servant_table['total_income'].mean()
print(servant_mean)

170898.30992266277


In [24]:
none_one_value = solvency.loc[solvency.loc[:,'income_type'] == 'предприниматель']
none_one_value

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,-520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


определил, что один пропуск в типе занятости "предприниматель". Т.к. предпринимателей всего 2, было решено - продублировать общий доход.

In [25]:
dubl = solvency[['income_type', 'total_income']]
dubl.loc[5936,'total_income'] = 499163
solvency.loc[5936, 'days_employed'] = 499163
solvency['total_income'] = dubl['total_income']
print(solvency.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19352 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19352 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [26]:
solvency.loc[solvency.loc[:,'income_type'] == 'сотрудник'] = solvency.loc[solvency.loc[:,'income_type'] == 'сотрудник'].fillna(worker_mean)
solvency.loc[solvency.loc[:,'income_type'] == 'компаньон'] = solvency.loc[solvency.loc[:,'income_type'] == 'компаньон'].fillna(companion_mean)
solvency.loc[solvency.loc[:,'income_type'] == 'пенсионер'] = solvency.loc[solvency.loc[:,'income_type'] == 'пенсионер'].fillna(pensioner_mean)
solvency.loc[solvency.loc[:,'income_type'] == 'госслужащий'] = solvency.loc[solvency.loc[:,'income_type'] == 'госслужащий'].fillna(servant_mean)

print(solvency.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Были обнаружены пропущенные значения в двух столбцах - 'total_income' и 'days_employed'.
Пропуски в столбце общего дохода были заполнены на средние значения в зависимости от типа занятости человека - рабочий, компаньон, пенсионер, студент, госслужащий. Один пропуск был заполнен индивидуально.
Т.к. столбец, отражающий общий стаж, в данном проекте не задействован, целесообразности работы с ним нет, пропуски были заполнены случайным образом, ради факта заполнения.

Причины пропусков в данных разные, возможно человеческие - должностное лицо не имело данной информации, не заполненность некоторых граф. Техниеские - утеряны при тех или иных манипуляциях.

### Замена типа данных

In [27]:
total_income_int = solvency['total_income']

In [28]:
solvency.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [29]:
solvency['total_income'] = solvency['total_income'].astype('int64')
solvency.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Заменён тип данных столбца 'total_income' с  float64 на int64, для экономии памяти, занимаемой таблицей.

### Обработка дубликатов

In [30]:
solvency = solvency.drop_duplicates().reset_index(drop = True)
solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Удалены дубликааты, которые могли возникнуть по человеческим причинам и техническим. Также могли сложиться обстоятельства, что все данные совпали у нескольких клиентов, что маловероятно и единично, что не влияет на общую картину.

### Лемматизация

In [31]:
from pymystem3 import Mystem
m = Mystem()

ModuleNotFoundError: No module named 'pymystem3'

In [32]:
purpose_table = solvency['purpose']
purpose_table.value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

определил цели кредитования. Выделил 5 категорий: свадьба, недвижимость, ремонт, автомобиль, образование.

Далее функция, которая проведет категоризацию в зависимости от цели получения кредита.

    категория 1 - это свадьба
    2 - недвжимость покупка
    3 - ремонт
    4 - все что связано с автомобилем, как правило покупка
    5 - образование

In [33]:
def lemmatize_purpose(row):
    row = m.lemmatize(row)
    for word in row:
        if 'свадьба' == word:
            return(1)
        elif ('недвижимость' == word) | ('жилье' == word) | ('жильё' == word):
            return(2)
        elif 'ремонт' == word:
            return(3)
        elif 'автомобиль' == word:
            return(4)
        elif 'образование' == word:
            return(5)


In [34]:
solvency['purpose_id'] = solvency['purpose'].apply(lemmatize_purpose)
solvency['purpose_id'].value_counts()

NameError: name 'm' is not defined

### Вывод

Были определены цели кредитования, лемматизация столбца 'purpose' и категоризация в зависимости от цели. Условное обозначение категории записано в новый столбец 'purpose_id'.

### Категоризация данных

Далее функция, которая категоризирует заемщиков на 2 группы - есть дети, нет детей. Чтобы сравнить платежеспособность этих 2х групп. Влияет ли факт наличия детей на выплаты кредита.

In [35]:
def children_not_yes(kid):
    if kid > 0:
        return('1')
    else:
        return('0')


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_id
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,1
21450,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,0
21451,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,1
21452,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,1


In [36]:
solvency['children_id'] = solvency['children'].apply(children_not_yes)
solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_id
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,1
21450,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,0
21451,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,1
21452,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,1


### Вывод

произведена категоризация заёмщиков в зависимости от наличия детей. Условное обозначение категории заёмщиков с детьми - "1", без детой - "0". Данные записаны в столбец 'children_id'. 

### Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [37]:
children_debt = solvency[['children', 'children_id', 'debt']]
children_debt

,children,children_id,debt
0,1,1,0
1,1,1,0
2,0,0,0
3,3,1,0
4,0,0,0
...,...,...,...
21449,1,1,0
21450,0,0,0
21451,1,1,1
21452,3,1,1


In [38]:
children_debt.corr(method='pearson')

,children,debt
children,1.000000,0.024686
debt,0.024686,1.000000


In [39]:
children_debt['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

In [40]:
children_debt['children_id'].value_counts()

0    14091
1     7363
Name: children_id, dtype: int64

In [47]:
children_debt.head()

,children,children_id,debt
0,1,1,0
1,1,1,0
2,0,0,0
3,3,1,0
4,0,0,0


Далее имею общее количество заёмщиков с детьми и без детей.

In [42]:
only_debt = children_debt.loc[children_debt.loc[:, 'debt'] == 1]
only_debt

,children,children_id,debt
14,0,0,1
32,0,0,1
38,0,0,1
55,0,0,1
75,1,1,1
...,...,...,...
21423,0,0,1
21440,0,0,1
21444,1,1,1
21451,1,1,1


In [43]:
only_debt['children_id'].value_counts()

0    1063
1     678
Name: children_id, dtype: int64

Здесь получил данные по количеству заёмшиков с долгом в зависимости от наличия детей.

Далее хочу посмотреть стандартизованную разницу между заёмщиками с долгом по категориям - с детьми и без. Для этого получу соотношение должников без детей к общему числу заёмщиков без детей, аналогично с детьми. На основании полученных данных сделаю вывод.

In [44]:
without_kids_debt = 1063 / 14091
without_kids_debt
print("Задолженность заёмщиков без детей = {:.1%}".format(without_kids_debt))

Задолженность заёмщиков без детей = 7.5%


In [45]:
with_kids_debt = 678 / 7363
print("Задолженность заёмщиков с детьми = {:.1%}".format(with_kids_debt))

Задолженность заёмщиков с детьми = 9.2%


### Вывод

Процент выплат среди заёмщиков с детьми выше относительно заёмщиков без детей на 1.7%. Корреляции между наличием детей и выплатой кредита нет. 
Следовательно значимой зависимости нет.

                                                        ***

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [53]:
solvency['family_status_id'].value_counts()

0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

In [54]:
family_debt = solvency[['family_status_id', 'debt']]


In [55]:
family_debt.corr()

,family_status_id,debt
family_status_id,1.000000,0.020347
debt,0.020347,1.000000


In [56]:
only_debt_family = family_debt.loc[family_debt.loc[:, 'debt'] == 1]
only_debt_family['family_status_id'].value_counts()

0    931
1    388
4    274
3     85
2     63
Name: family_status_id, dtype: int64

In [57]:
married_debt = 931 / 12339
civil_marriage = 388 / 4151
widow = 63 / 959
divorce = 85 / 1195
single = 274 / 2810
print("Задолженность в зависимости от семейного положения")
print("женатые/замужем = {:.1%}".format(married_debt))
print("в гражданском браке (сожители) = {:.1%}".format(civil_marriage))
print("вдовствующие = {:.1%}".format(widow))
print("разведённые = {:.1%}".format(divorce))
print("одинокие = {:.1%}".format(single))

Задолженность в зависимости от семейного положения
женатые/замужем = 7.5%
в гражданском браке (сожители) = 9.3%
вдовствующие = 6.6%
разведённые = 7.1%
одинокие = 9.8%


### Вывод

Видим, что у женатых, в разводе и вдовцов самые высокие выплаты по кредитованию. У одиноких и живущих в гражданском браке (сожители) невыплаты по кредиту составляют примерно одинаковый уровень, который выше других категорий граждан по семейному положению.

Тут можно предложить в качестве продолжения анализа посмотреть зависимость задолженности от возраста, чтобы проверить гипотезу - чем старше (до определённого возраста), тем надёжнее заёмщик. Основание для такой гипотезы - вдовцы, как самая надёжная категория, возможно уже находятся в среднем возрасте или пенсионном, также как и разведённые. А среди одиноких и пребывающих в гражданском браке (сожителей) - бОльшая часть - люди молодого возраста. Посмотреть привязку возрастных периодов к семейному положению.

                                                        ***

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [62]:
income_debt = solvency[['total_income', 'debt']]


In [63]:
income_debt.corr()

,total_income,debt
total_income,1.000000,-0.012854
debt,-0.012854,1.000000


### Вывод

Статистически значимой корреляции между доходом и выплатой кредита нет. Возможно это связано с суммой кредитования. Заёмщики с бОльшим доходом берут бОльшие суммы займа.

                                                        ***

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_debt = solvency[['purpose_id', 'debt']]
purpose_debt['purpose_id'].value_counts()

In [ ]:
purpose_debt.corr()

In [ ]:
only_purpose_debt = purpose_debt.loc[purpose_debt.loc[:, 'debt'] == 1]
only_purpose_debt['purpose_id'].value_counts()

In [ ]:
wedding = 186 / 2324
flat = 747 / 10204
repairs = 35 / 607
car = 403 / 4306
education = 370 / 4013
print("Задолженность в зависимости от цели кредитования")
print("недвижимость = {:.1%}".format(flat))
print("автомобиль = {:.1%}".format(car))
print("образование = {:.1%}".format(education))
print("свадьба = {:.1%}".format(wedding))
print("ремонт = {:.1%}".format(repairs))

### Вывод

Люди, берущие кредит для приобретения, строительства или ремонта недвижимости, возвращают долг с большей вероятностью нежели люди, берущие кредит для приобретения автомобиля, свадьбы или оплаты образовательных услуг.

### Шаг 4. Общий вывод

Зависимость показателя своевременной и добросовестной выплаты долга весьма полиэтиологична. Влияет множество факторов: семейное положение, тип занятости, наличие детей, цели кредитования и прочее, в т.ч., возможно, и экономическая стабильность страны. Остаётся непроверенными и другие факторы, к примеру - образование заёмщика, возможно оно коррелирует с социальной ответственностью, лучшим планированием и возвратом долга. 

Эти данные могут помочь оценить банку риски, помочь принять решение о выдаче кредита, сумме заёма и верное решение - поможет защитить заёмщика от долга.